# Доп занятие, основы веб скрейпинга

## web scraping

Откуда можно брать данные?

* https://toolbox.google.com/datasetsearch
* https://www.kaggle.com/
* Найти данные в сети и пропарсить их (Beautiful Soup)

### kaggle

Пример данных:

https://www.kaggle.com/dgomonov/data-exploration-on-nyc-airbnb

### Основы HTML

* [Подробнее](http://www.astro.spbu.ru/staff/afk/Teaching/Help/StrHTML.htm)
* [Набор полезных материалов](http://www.astro.spbu.ru/staff/afk/Teaching/Help/HTMLref.htm)

Перед тем как анализировать страницы в web нужно понять на чем они написаны. Большинство веб-страниц имеют содержание разметки на языке HTML (или XHTML).
С помощью HTML можно легко создать относительно простой, но красиво оформленный документ. Помимо упрощения структуры документа, в HTML внесена поддержка гипертекста.


Документ HTML включает в себя управляющие команды (иначе называемые операторами или тегами) и сам форматируемый с помощью тегов текст документа. Теги выделяются угловыми скобками  < и >.   Первым тегом, с которого следует начинать описание документа HTML является тег `<HTML>` , открывающий документ. Завершает описание документа тег  `</HTML>`.

Давайте посмотрим на [примеры тегов](http://www.astro.spbu.ru/staff/afk/Teaching/Help/Tegs.htm)




### Семантическая вёрстка

![мемчики](http://lurkmore.so/images/b/b7/Publish_mudaki_2.png)

Семантическая вёрстка, или семантический HTML-код, — это подход к созданию веб-страниц на языке HTML, основанный на использовании HTML-тегов в соответствии с их семантикой (предназначением), а также предполагающий логичную и последовательную иерархию страницы. Он противопоставляется подходу, при котором написание HTML-кода определяется внешним видом веб-страницы. Для оформления веб-страниц, написанных в соответствии с семантикой, используются каскадные таблицы стилей (CSS). Стандарт HTML с самого начала включал в себя ряд семантических тегов, но большую популярность семантическая вёрстка получила после начала работ над HTML5.



### Как может выглядеть задача по плану плане?

![Как выглядит задача](https://raw.githubusercontent.com/tixonsit/Sberbank_materials/master/1.jpg)

### Как этот код может выглядеть в реальной жизни в html?

![somebody was told me the world is gonna roll me](https://raw.githubusercontent.com/tixonsit/Sberbank_materials/master/2.png)

### Beautiful Soup

На основе [официальной документации](http://wiki.python.su/%D0%94%D0%BE%D0%BA%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D0%B8/BeautifulSoup#A.2BBBEESwRBBEIEQARLBDk_.2BBEEEQgQwBEAEQg-)

Beautiful Soup - это парсер для синтаксического разбора файлов HTML/XML, написанный на языке программирования Python, который может преобразовать даже неправильную разметку в дерево синтаксического разбора. Он поддерживает простые и естественные способы навигации, поиска и модификации дерева синтаксического разбора.

In [ ]:
# установка
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup("<p>Привет<b>всем<i>HTML")

print(soup.prettify())

<html>
 <body>
  <p>
   Привет
   <b>
    всем
    <i>
     HTML
    </i>
   </b>
  </p>
 </body>
</html>


In [ ]:
from bs4 import BeautifulSoup
import re

doc = ['<html><head><title>Page title</title></head>',
       '<body><p id="firstpara" align="center">This is paragraph <b>one</b>.',
       '<p id="secondpara" align="blah">This is paragraph <b>two</b>.',
       '</html>']
soup = BeautifulSoup(''.join(doc))

print(soup.prettify())

<html>
 <head>
  <title>
   Page title
  </title>
 </head>
 <body>
  <p align="center" id="firstpara">
   This is paragraph
   <b>
    one
   </b>
   .
  </p>
  <p align="blah" id="secondpara">
   This is paragraph
   <b>
    two
   </b>
   .
  </p>
 </body>
</html>


### Навигация по супу:

In [ ]:
head = soup.contents[0].contents[0]

In [ ]:
head.text

'Page title'

In [ ]:
titleTag = soup.head.title
titleTag

<title>Page title</title>

In [ ]:
head.nextSibling.contents[0]

<p align="center" id="firstpara">This is paragraph <b>one</b>.</p>

In [ ]:
head.nextSibling.contents[0].nextSibling

<p align="blah" id="secondpara">This is paragraph <b>two</b>.</p>

In [ ]:
soup.find('p').b.string

'one'

In [ ]:
soup('p')[1].b.string

'two'

### Практический пример

Предположим, что нам нужно создадим базу данных (БД) текстов по определенной тематике для задачи анализа текстов. Как ее можно решить? Найдем сайт на котором реализован поиск по ключевым словам и будем его анализировать. В нашем примере будем использовать сайт arXiv.org — крупнейший бесплатный архив электронных публикаций научных статей и их препринтов по физике, математике, астрономии, информатике и биологии.

Создадим наше БД в виде словаря.

{ ключ : значение } - формат словаря.

Ключем будет заголовок статьи. Значением будет краткое описание (abstract) статьи.



Для начала определим откуда мы будем брать наши данные?

Посмотрим на ссылку из поска по сайту архива:

https://arxiv.org/search/?query=data+analysis&searchtype=all&source=header

Можно видеть, что наш запрос "data analysis" выглядит внутри ссылки как "data+analysis"



#### Выделение url запроса

In [ ]:
import urllib.request
from bs4 import BeautifulSoup

# искомая фраза
searched_phrase = 'data analysis'

# заменим пробел в искомой фразе на + для соответсвия формату запроса
searched_phrase = searched_phrase.replace(' ', '+')

# что может представлять из себя реальная html страница?
# подставим в поиск по сайту искомую фразу
page = urllib.request.urlopen(f"https://arxiv.org/search/?query={searched_phrase}&searchtype=all&source=header")

# создаем красивый суп
soup = BeautifulSoup(page)
# print(soup.prettify())

#### Выделение искомого объекта

Теперь нам надо определить что мы ищем? Текст статьи и ее заголовок находятся в теге li, в классе под названием arxiv-result. Дадим beautiful soup самостоятельно найти статью всего в одну комманду.
Метод обхода дерева findAll начинает с заданной точки и ищет все объекты Tag и NavigableString, соответствующие заданным критериям. Сигнатура метода findall следующая:

```
soup.findAll(name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs)
```
В нашем случае мы ищем тег li и указываем наличие атрибута class с названием arxiv-result
```
soup.find_all('li', {'class': 'arxiv-result'})
```



In [ ]:
print(len(
        list(
            soup.find_all('li', {'class': 'arxiv-result'})
            )
        )
    )

50


Отобразим, что из себя представляет найденный объект

In [ ]:
print(list(
            soup.find_all('li', {'class': 'arxiv-result'})
            )[0]
        )

<li class="arxiv-result">
<div class="is-marginless">
<p class="list-title is-inline-block"><a href="https://arxiv.org/abs/2202.12875">arXiv:2202.12875</a>
<span> [<a href="https://arxiv.org/pdf/2202.12875">pdf</a>, <a href="https://arxiv.org/format/2202.12875">other</a>] </span>
</p>
<div class="tags is-inline-block">
<span class="tag is-small is-link tooltip is-tooltip-top" data-tooltip="Machine Learning">cs.LG</span>
<span class="tag is-small is-grey tooltip is-tooltip-top" data-tooltip="Artificial Intelligence">cs.AI</span>
<span class="tag is-small is-grey tooltip is-tooltip-top" data-tooltip="Computation and Language">cs.CL</span>
</div>
</div>
<p class="title is-5 mathjax">
      
        DataLab: A Platform for <span class="search-hit mathjax">Data</span> <span class="search-hit mathjax">Analysis</span> and Intervention
      
    </p>
<p class="authors">
<span class="has-text-black-bis has-text-weight-semibold">Authors:</span>
<a href="/search/?searchtype=author&amp;query=Xiao

#### Оптимизация запроса

Можно видеть, что всего у нас 50 объектов. Для того, чтоб не перегружать сервера частыми запросами (чтоб нас не забанили) оптимизируем запрос уже на этом этапе - будем отображать сразу 200 статей на странице.

Запрос изменится на:

https://arxiv.org/search/?searchtype=all&query=data+analysis&abstracts=show&size=200&order=-announced_date_first

In [ ]:
# искомая фраза
searched_phrase = 'data analysis'

# заменим пробел в искомой фразе на + для соответсвия формату запроса
searched_phrase = searched_phrase.replace(' ', '+')

# что может представлять из себя реальная html страница?
# подставим в поиск по сайту искомую фразу
page = urllib.request.urlopen(f"https://arxiv.org/search/?searchtype=all&query={searched_phrase}&abstracts=show&size=200&order=-announced_date_first")

# создаем красивый суп
soup = BeautifulSoup(page)
# print(soup.prettify())

Теперь можно убедиться, что их стало 200, тем самым мы существенно уменьшили кол-во запросов на этапе предобработки.

In [ ]:
print(len(
        list(
            soup.find_all('li', {'class': 'arxiv-result'})
            )
        )
    )

200


#### Выделение внутренних компонент

Теперь давайте попробуем проитерироваться по нашему объекту для выделения необходимых компонентов. В последсвии во внутренних поисках мы будем итерироваться по аналогии.

In [ ]:
# Посмотрим на тип данных одного litag
print(type(soup.find_all('li', {'class': 'arxiv-result'})))

<class 'bs4.element.ResultSet'>


In [ ]:
for litag in soup.find_all('li', {'class': 'arxiv-result'}):
    # результат нашего запроса bs4.element.ResultSet можно преобразовать в текст используя .text :
    print(litag.text)
    # break
    # уже можно видеть, что мы на верном пути
    
    print(len(litag.text))
    break



arXiv:2202.12875
 [pdf, other] 


cs.LG
cs.AI
cs.CL



      
        DataLab: A Platform for Data Analysis and Intervention
      
    

Authors:
Yang Xiao, 
      
      Jinlan Fu, 
      
      Weizhe Yuan, 
      
      Vijay Viswanathan, 
      
      Zhoumianze Liu, 
      
      Yixin Liu, 
      
      Graham Neubig, 
      
      Pengfei Liu


Abstract:
      
        Despite data's crucial role in machine learning, most existing tools and research tend to focus on systems on top of existing…
        ▽ More


        Despite data's crucial role in machine learning, most existing tools and research tend to focus on systems on top of existing data rather than how to interpret and manipulate data. In this paper, we propose DataLab, a unified data-oriented platform that not only allows users to interactively analyze the characteristics of data, but also provides a standardized interface for different data processing operations. Additionally, in view of the ongoing proliferation 

#### Заголовок

Теперь нам необходимо найти внутри полученного объекта заголовок и краткое описание. Используя браузер можно увидеть, что title содержится внутри нашего li тега в теге p, а именно в классе title is-5 mathjax

По аналогии с предыдущим примером распакуем уже наш литаг поиском:
```
litag.find_all('p', {'class': 'title is-5 mathjax'})
```



In [ ]:
for litag in soup.find_all('li', {'class': 'arxiv-result'}):
    for title in litag.find_all('p', {'class': 'title is-5 mathjax'}):
        # отобразим название статьи (преобразовав в текст)
#         print(title.text)
        
        # обрезаем пробелы, теперь это уже можно использовать как ключ
        print(title.text[16:-12])

        # для красоты вывода отобразим только один элемент - для этого break дважды
        break
    break

DataLab: A Platform for Data Analysis and Intervention


#### Описание

Осталось только найти описание (abstract) по аналогии внутри тега li. Используя браузер можно увидеть, что abstract содержится внутри тега li внутри тега span, а именно в классе abstract-full has-text-grey-dark mathjax

По аналогии с предыдущим примером распакуем уже наш литаг поиском:
```
litag.find_all('span', {'class': 'abstract-full has-text-grey-dark mathjax'})
```



In [ ]:


for litag in soup.find_all('li', {'class': 'arxiv-result'}):
    for abstract in litag.find_all('span', {'class': 'abstract-full has-text-grey-dark mathjax'}):
        # отобразим описание статьи (преобразовав в текст)
#         print(abstract.text)

        # обрезаем пробелы, теперь это уже можно использовать как значение
        print(abstract.text[9:-16])

        # для красоты вывода отобразим только один элемент - для этого break дважды
        break
    break

Despite data's crucial role in machine learning, most existing tools and research tend to focus on systems on top of existing data rather than how to interpret and manipulate data. In this paper, we propose DataLab, a unified data-oriented platform that not only allows users to interactively analyze the characteristics of data, but also provides a standardized interface for different data processing operations. Additionally, in view of the ongoing proliferation of datasets, \toolname has features for dataset recommendation and global vision analysis that help researchers form a better view of the data ecosystem. So far, DataLab covers 1,715 datasets and 3,583 of its transformed version (e.g., hyponyms replacement), where 728 datasets support various analyses (e.g., with respect to gender bias) with the help of 140M samples annotated by 318 feature functions. DataLab is under active development and will be supported going forward. We have released a web platform, web API, Python SDK, Py

#### Сведение всех запросов, формирование БД

Мы нашли название и описание, давайте посмотрим на код вместе:

In [ ]:
for litag in soup.find_all('li', {'class': 'arxiv-result'}):
    # заголовок
    for title in litag.find_all('p', {'class': 'title is-5 mathjax'}):
        title = title.text[16:-12]

    # описание
    for abstract in litag.find_all('span', {'class': 'abstract-full has-text-grey-dark mathjax'}):
        summary = abstract.text[9:-16]

    # отображение
    print('Название статьи:\n', title)
    print()
    print('Текст статьи:\n', summary)
    break

Название статьи:
 DataLab: A Platform for Data Analysis and Intervention

Текст статьи:
 Despite data's crucial role in machine learning, most existing tools and research tend to focus on systems on top of existing data rather than how to interpret and manipulate data. In this paper, we propose DataLab, a unified data-oriented platform that not only allows users to interactively analyze the characteristics of data, but also provides a standardized interface for different data processing operations. Additionally, in view of the ongoing proliferation of datasets, \toolname has features for dataset recommendation and global vision analysis that help researchers form a better view of the data ecosystem. So far, DataLab covers 1,715 datasets and 3,583 of its transformed version (e.g., hyponyms replacement), where 728 datasets support various analyses (e.g., with respect to gender bias) with the help of 140M samples annotated by 318 feature functions. DataLab is under active development and 

Отсалось только записать в базу данных

In [ ]:
data_base = {}
for litag in soup.find_all('li', {'class': 'arxiv-result'}):
    for title in litag.find_all('p', {'class': 'title is-5 mathjax'}):
        title = title.text[16:-12]

    for abstract in litag.find_all('span', {'class': 'abstract-full has-text-grey-dark mathjax'}):
        summary = abstract.text[9:-16]

    # как добавить элемент в словарь? <название словаря>[ключ] = значение
    data_base[title] = summary

# отобразим размер отпаршеной страницы
print(len(data_base))

200


#### Корректность заполнения

Проверим все ли хорошо заполнилось? Отобразим 5 случайных значений словаря

In [ ]:
for i in range(5):
    # используем .items() для доступа к значениям
    # преобразовываем в список
    # отображаем i-й элемент (первые 5)
    title, summary = list(data_base.items())[i]

    print('ПРИМЕР №', i+1)
    print('Название статьи:\n', title)
    print('Текст статьи:\n', summary)
    print()

ПРИМЕР № 1
Название статьи:
 DataLab: A Platform for Data Analysis and Intervention
Текст статьи:
 Despite data's crucial role in machine learning, most existing tools and research tend to focus on systems on top of existing data rather than how to interpret and manipulate data. In this paper, we propose DataLab, a unified data-oriented platform that not only allows users to interactively analyze the characteristics of data, but also provides a standardized interface for different data processing operations. Additionally, in view of the ongoing proliferation of datasets, \toolname has features for dataset recommendation and global vision analysis that help researchers form a better view of the data ecosystem. So far, DataLab covers 1,715 datasets and 3,583 of its transformed version (e.g., hyponyms replacement), where 728 datasets support various analyses (e.g., with respect to gender bias) with the help of 140M samples annotated by 318 feature functions. DataLab is under active develo

#### Автоматизация, нахождение количества страниц

У нас есть основа для анализа одной страницы, хорошо было бы отпарсить много разных запросов (не только data analysis) и парсить не первые 200 запросов, а все доступные страницы.

Каким образом можно найти все доступные страницы? Опять обратимся к встроенному в браузер редактору через F12.

Количество отображено в теге h1, а именно в классе title is-clearfix. По аналогии со всеми примерами до этого отобразим его.

In [ ]:
soup.find_all('h1', {'class': 'title is-clearfix'})

[<h1 class="title is-clearfix">
     
         Showing 1–200 of 101,328 results for all: <span class="mathjax">data analysis</span>
 </h1>]

Искомый кусок текста - 73,670 - его необходимо преобразовать в int 73670

In [ ]:
# элемент всего 1, обратимся к нему через [0], преобразуем в текст
ammount = soup.find_all('h1', {'class': 'title is-clearfix'})[0].text

Теперь осталось найти само число - оно стоит после 'of '
Вспоминаем первые занятия и str:

In [ ]:
ammount.split('of ')

['\n    \n        Showing 1–200 ', '101,328 results for all: data analysis\n']

Получили на выход list из 2х элементов, искомое значение во 2м, обратимся к нему по индексу [1] и обрежем до 6 знака

In [ ]:
ammount = ammount.split('of ')[1][:6]
ammount

'101,32'

Знаков может быть произвольное количество, в зависимости от количества доступных статей:

In [ ]:
'''
Такой код не является оптимальным! Это костыли!
Желательно использовать библиотеку re для поиска паттернов!
'''

try:
    # до запятой - тясячи, опять перепишем это в переменные тысяч и остатка
    thousands, num = ammount.split(',')
    # приводим к типу int, умножаем, объединяем с остатком
    ammount = int(thousands)*1000 + int(num)
except:
    # если статей слишком мало - запятой не будет, в таком случае исходный текст
    # необходимо обрезать еще на пару знаков и привести из str в int
    ammount = int(ammount[:-2])

print(ammount)

101032


Итого итоговый код для поиска кол-ва статей по запросу

In [ ]:
import math

ammount = 0
for articles_ammount in soup.find_all('h1', {'class': 'title is-clearfix'}):      
    amm_text = articles_ammount.text.split('of ')[1][:6]
    try:
        thousands, num = amm_text.split(',')
        ammount = int(thousands)*1000 + int(num)
    except:
        ammount = int(amm_text[:-2])

print('Количество статей', ammount)
# Поскольку мы будем грузить по 200 статей на странице нам нужно найти остаток от
# деления количества статей на 200
print('Количество страниц для парсинга', math.ceil(ammount/200))

Количество статей 101032
Количество страниц для парсинга 506


#### Автоматизация запроса под количество доступных статей

Будем проходиться по всем страницам т.к. мы уже знаем какое их количество, для этого посмотрим как выглядит url на 2й странице:

https://arxiv.org/search/?searchtype=all&query=data+analysis&abstracts=show&size=200&order=-announced_date_first&date-date_type=submitted_date&start=200

Можно увидеть, что в конце есть параметр start=200, он показывает, с какой статьи отображать. Нам осталось только пройтись в цикле по всем страницам. 

!!! На это занятии я специально поставлю break, чтоб не устроить ddos атаку на сервера архива. !!!

In [ ]:
import math
from tqdm import tqdm

searched_phrase = 'data analysis'

# заменим пробел в искомой фразе на + для соответсвия формату запроса
searched_phrase = searched_phrase.replace(' ', '+')

ammount = 0
for articles_ammount in soup.find_all('h1', {'class': 'title is-clearfix'}):      
    amm_text = articles_ammount.text.split('of ')[1][:6]
    try:
        thousands, num = amm_text.split(',')
        ammount = int(thousands)*1000 + int(num)
    except:
        ammount = int(amm_text[:-2])

print('Количество статей', ammount)
# Поскольку мы будем грузить по 200 статей на странице нам нужно найти остаток от
# деления количества статей на 200
print('Количество страниц для парсинга', math.ceil(ammount/200))
pages_ammount = math.ceil(ammount/200)

for i in tqdm(range(pages_ammount)):
    url = f'https://arxiv.org/search/?searchtype=all&query={searched_phrase}&abstracts=show&size=200&order=-announced_date_first&start={i*200}'
    # print(f'Обрабатываем страницы с {i*200} по {(i+1)*200} из {ammount}')

    # break

Количество статей 97083
Количество страниц для парсинга 486


100%|██████████| 486/486 [00:00<00:00, 921950.13it/s]


#### Объединяем все воедино

In [ ]:
import math

searched_phrase = 'data analysis'

# заменим пробел в искомой фразе на + для соответсвия формату запроса
searched_phrase = searched_phrase.replace(' ', '+')

# находим количество статей
ammount = 0
for articles_ammount in soup.find_all('h1', {'class': 'title is-clearfix'}):      
    amm_text = articles_ammount.text.split('of ')[1][:6]
    try:
        thousands, num = amm_text.split(',')
        ammount = int(thousands)*1000 + int(num)
    except:
        ammount = int(amm_text[:-2])

print('Количество статей', ammount)
# Поскольку мы будем грузить по 200 статей на странице нам нужно найти остаток от
# деления количества статей на 200
print('Количество страниц для парсинга', math.ceil(ammount/200))
pages_ammount = math.ceil(ammount/200)

# База данных
data_base = {}

# проходимся по всем страицам
for i in range(pages_ammount):
    url = f'https://arxiv.org/search/?searchtype=all&query={searched_phrase}&abstracts=show&size=200&order=-announced_date_first&start={i*200}'
    print(f'Обрабатываем статьи с {i*200} по {(i+1)*200} из {ammount}')

    # подставим в поиск по сайту искомую фразу
    page = urllib.request.urlopen(url)

    # создаем красивый суп
    soup = BeautifulSoup(page)

    # находим сам блок, описывающий статью
    for litag in soup.find_all('li', {'class': 'arxiv-result'}):
        # находим заголовок
        for title in litag.find_all('p', {'class': 'title is-5 mathjax'}):
            title = title.text[16:-12]

        # находим описание
        for abstract in litag.find_all('span', {'class': 'abstract-full has-text-grey-dark mathjax'}):
            summary = abstract.text[9:-16]

        # как добавить элемент в словарь? <название словаря>[ключ] = значение
        data_base[title] = summary

    # отпарсим первые 2 страницы
    if i == 1:
        break

Количество статей 101032
Количество страниц для парсинга 506
Обрабатываем статьи с 0 по 200 из 101032
Обрабатываем статьи с 200 по 400 из 101032


In [ ]:
# отобразим размер отпаршеной страницы
print(len(data_base))

400


In [ ]:
for i in range(200, 205):
    # используем .items() для доступа к значениям
    # преобразовываем в список
    # отображаем i-й элемент (первые 5)
    title, summary = list(data_base.items())[i]

    print('ПРИМЕР №', i+1)
    print('Название статьи:\n', title)
    print('Текст статьи:\n', summary)
    print()

ПРИМЕР № 201
Название статьи:
 On the Dynamics of Solid, Liquid and Digital Gold Futures
Текст статьи:
 This paper examines the determinants of the volatility of futures prices and basis for three commodities: gold, oil and bitcoin -- often dubbed solid, liquid and digital gold -- by using contract-by-contract analysis which has been previously applied to crude oil futures volatility investigations. By extracting the spot and futures daily prices as well as the maturity, trading volume and open interest data for the three assets from 18th December 2017 to 30th November 2021, we find a positive and significant role for trading volume and a possible negative influence of open interest, when significant, in shaping the volatility in all three assets, supporting earlier findings in the context of oil futures. Additionally, we find maturity has a relatively positive significance for bitcoin and oil futures price volatility. Furthermore, our analysis demonstrates that maturity affects the ba

#### Проблемы подхода?

1.   Вас забанят по ip, используйте прокси
2.   Поведение роботов легко отследить, для этого есть капча --> решается нейросетями

